In [1]:
""" 
Assessing greenspace accessability within Northern Ireland through an interactive map which calculates the nearest proximity of a settlement to a greenspace trail. 
"""
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely.strtree import STRtree
from shapely.ops import nearest_points

In [10]:
"""
Load settlement, trail, and greenspace shapefiles.
"""
settlements = gpd.read_file("data/Settlements/settlements-2015-above-500-threshold.shp").to_crs(epsg=3857)
trails = gpd.read_file("data/Trails/greenspaceoffroadtrails.shp").to_crs(epsg=3857)
greenspace = gpd.read_file("data/Greenspace/greenspace.shp").to_crs(epsg=3857)


In [11]:
"""
Check the EPSG of each data file.
"""
settlements.crs
trails.crs
greenspace.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
"""
Calculate area (in hectares) for each greenspace polygon and store its centroid
for labeling or proximity operations.
"""
if 'Area_Ha' not in greenspace.columns:
    greenspace['Area_Ha'] = greenspace.geometry.area / 10_000  # Convert m² to ha

greenspace['centroid'] = greenspace.geometry.centroid

In [13]:
"""
Use a spatial index (STRtree) to efficiently find the nearest trail segment
to each settlement. Record both the connecting line and the distance.
"""
trail_geoms = trails.geometry.tolist()
trail_tree = STRtree(trail_geoms)

connection_lines = []  # List to hold LineStrings from settlements to trails
distances = []         # List to hold Euclidean distances (in meters)

for idx, row in settlements.iterrows():
    point = row.geometry.centroid
    nearest_idx = trail_tree.nearest(point)
    nearest_geom = trail_geoms[nearest_idx]
    nearest_point = nearest_points(point, nearest_geom)[1]

    line = LineString([point, nearest_point])
    distance = point.distance(nearest_point)

    connection_lines.append(line)
    distances.append(distance)

# Create GeoDataFrame of connection lines and distances
connections = gpd.GeoDataFrame(
    {'distance_m': distances},
    geometry=connection_lines,
    crs=settlements.crs
)

In [14]:
"""
Check the EPSG of the connections.gpd.
"""
connections.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich